In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

import os
import time
import pickle
import math
import itertools
import copy

In [3]:
# Device config
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper params
batch_size = 32
num_epochs = 50
learning_rate = 0.001
momentum = 0.9

# Random seed
seed = 2022
torch.random.manual_seed(seed)
np.random.seed(2022)

# Tensorboard writer
# writer = SummaryWriter("runs/multilabelsclass")

# logger
logger = TensorBoardLogger('runs', 'lightning')

# Data Module

# 1. Read Image & Transform

In [4]:
# Data directory
data_dir = '../pascal_2007'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
    
# Save directory
save_dir = './saved'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# Image directory
train_img_dir = os.path.join(data_dir, 'train')
test_img_dir = os.path.join(data_dir, 'test')

# Annotaion file
train_anno_file = os.path.join(data_dir, 'train.csv')
test_anno_file = os.path.join(data_dir, 'test.csv')

In [5]:
## Create custom dataset:

class ImageDataset(Dataset):
    # __init_function run once when instantiating the Dataset object
    def __init__(self, annotations, images, labellist, transforms=None):
        super().__init__()
        self.img = images
        self.annotations = annotations    
        self.labellist = labellist
        self.transforms = transforms
        
    # __len__ func return the num of samples in dataset 
    def __len__(self):
        return len(self.annotations)
    
    # Load and return a sample from the dataset at the given index `idx` then convert it to tensor 
    # retrieves the corresponding labels from the csv data, call the transform function on them (if applicable)
    # then return the tensor image and corresponding label.
    
    # convert label of and sample to index of these label in the labellist
    def text_to_index(self, labellist, annotations):
        index = []
        for item in annotations:
            index.append(self.labellist.index(item))
        return index
    
    def __getitem__(self, idx):
        
        # convert image to tensor and transform image
        image = self.img[idx]
        if self.transforms:
            image = self.transforms(image)
            
        # convert label to one hot encoding
        labels_index = self.text_to_index(self.labellist, self.annotations[idx])
        labels_index = torch.tensor(labels_index)
        labels_onehot = F.one_hot(labels_index, num_classes=len(self.labellist))
        onehot_label = labels_onehot.sum(dim=0).float()
        
        return image, onehot_label

In [7]:
# Data transform and argumentation (data preprocessing)

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),        
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Normalize(mean, std)
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Normalize(mean, std)
    ])
}

In [26]:
class PascalDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, save_dir, transforms, batch_size: 32):
        super().__init__()
        self.data_dir = data_dir
        self.train_img_dir = os.path.join(self.data_dir, 'train') 
        self.test_img_dir = os.path.join(self.data_dir, 'test')
        self.train_anno_file = os.path.join(self.data_dir, 'train.csv')
        self.test_anno_file = os.path.join(self.data_dir, 'test.csv')
        self.save_dir = save_dir
        
        self.batch_size = batch_size
        self.transforms = transforms
        
    def setup(self, stage: None):
        (train_images, training_labels), (val_images, validation_labels) = self.load_train_images_and_labels(self.train_img_dir, self.train_anno_file)
        (test_images, test_labels) = self.load_test_images_and_labels(self.test_img_dir, self.test_anno_file)
        
        # labels
        self.labels = self.get_labels(training_labels)
        
        self.images_pascal = {
        'train' : ImageDataset(training_labels, train_images, self.labels, data_transforms['train']),
        'val': ImageDataset(validation_labels, val_images, self.labels, data_transforms['val']),
        'test': ImageDataset(test_labels, test_images, self.labels, data_transforms['test'])
        }
    
    def train_dataloader(self):
        return DataLoader(self.images_pascal['train'], batch_size=16, shuffle=True)
    def val_dataloader(self):
        return DataLoader(self.images_pascal['val'], batch_size=16, shuffle=False)
    def test_dataloader(self):
        return DataLoader(self.images_pascal['test'], batch_size=16, shuffle=False)
    
    def get_labels(self, training_labels):
        # Get all labels
        labels = []

        for label in training_labels:
            labels = labels + label

        labels = sorted(list(set(labels)))

        print('Number of labels: ', len(labels))
        return labels
    
    def convert_label_to_array(self, labels):
        for ind, label in enumerate(labels):
            labels[ind] = label.split(" ")
        return labels    

    def load_train_images_and_labels(self, image_dir, anno_path):
        train_label = pd.read_csv(anno_path)
        train_image_label_value = (train_label["is_valid"].values)
        
        # get all index of training image and validation image
        train_image_label_index = [i for i, x in enumerate(train_image_label_value) if x]
        valid_image_label_index = [i for i, x in enumerate(train_image_label_value) if not x]
        
        ## get all image file
        image_file_name = train_label["fname"].values
        
        # Read all training image
        total_train_image = [] 
        for image in image_file_name:
            image = plt.imread(os.path.join(image_dir, image))
            total_train_image.append(image)
        
        train_images = [ total_train_image[i] for i in train_image_label_index]
        val_images = [ total_train_image[i] for i in valid_image_label_index]
        
        ## Read all label for train and val data
        labels = train_label["labels"].values

        training_labels = labels[train_image_label_index]
        validation_labels = labels[valid_image_label_index]
        
        training_labels = self.convert_label_to_array(training_labels)
        validation_labels = self.convert_label_to_array(validation_labels)
        
        return (train_images, training_labels), (val_images, validation_labels)
   
    def load_test_images_and_labels(self, image_dir, anno_path):
        ## get all image file
        test_label = pd.read_csv(anno_path)
        test_image_file_name = test_label["fname"].values
        
        ## Test data
        test_images = [] 
        for image in test_image_file_name:
            image = plt.imread(os.path.join(image_dir, image))
            test_images.append(image)

        test_labels = test_label["labels"].values
        test_labels = self.convert_label_to_array(test_labels)
        
        return (test_images, test_labels)
    

In [27]:
data_module = PascalDataModule(data_dir, save_dir, transforms, batch_size)

## Lightning Module

In [12]:
## Load a pretrained model and reset the final fully connected layers
## FIne tuning the convnet: Instead of random initialization, we initialize the network with a pretrained network
from torch.optim import lr_scheduler

model = models.resnet50(pretrained=True)

class NeuralNetwork(nn.Module):
    def __init__(self, model, num_classes):
        super().__init__()
        self.model = model
        self.num_classes = num_classes
        
        #Finetuning the convnet
        in_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(in_ftrs, 128)
        self.fc_head = nn.Linear(128, self.num_classes)
        
    def forward(self, x):
        x = F.relu(self.model(x))
        x = torch.sigmoid(self.fc_head(x))
        return x

/home/longvuduc/DS_internship_2022/labtest/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/longvuduc/DS_internship_2022/labtest/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
class LitModel(pl.LightningModule):
    def __init__(self,
                num_classes,
                model,
                learning_rate=0.01,
                momentum=0.9,
                step_size=5,
                gamma=0.5):
        super().__init__()
        self.num_classes = num_classes
        self.model = NeuralNetwork(model, num_classes)
        
        # Hyper parameters
        self.learning_rate = learning_rate
        self.criterion = nn.BCELoss()
        self.momentum = momentum
        self.lr_schedule_step = step_size
        self.lr_schedule_factor = gamma
    
    def forward(self, x):
        x = self.model(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        
        preds = y_hat.round()
        num_corrects = torch.sum(preds == y.data, dim=0)
        cls_accuracy = num_corrects / y.size(0)
        avg_accuracy = cls_accuracy.mean() * 100
        
        tensorboard_log = {'acc': avg_accuracy}
        
        return {
            'loss': loss,
            'log': tensorboard_log
        }
    
        # Config callbacks
    def configure_callbacks(self):
        return [
            EarlyStopping(monitor='val/accuracy', mode='max', verbose=1, patience=10), 
            ModelCheckpoint(os.path.join(save_dir, 'checkpoints'), monitor='val/accuracy', mode='max', save_weights_only=True, verbose=1)
        ]
    
    def configure_optimizers(self, ):
        optimizer = optim.SGD(self.parameters(), lr=self.learning_rate, momentum=self.momentum)
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=self.lr_schedule_step, gamma=self.lr_schedule_factor)
        return [optimizer], [lr_scheduler]
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        preds = y_hat.round()
        num_corrects = torch.sum(preds == y.data, dim=0)
        cls_accuracy = num_corrects / y.size(0)
        avg_accuracy = cls_accuracy.mean() * 100
        
        tensorboard_log = {'acc': avg_accuracy}
        
        return {
            'loss': loss,
            'log': tensorboard_log
        }
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        preds = y_hat.round()
        num_corrects = torch.sum(preds == y.data, dim=0)
        cls_accuracy = num_corrects / y.size(0)
        avg_accuracy = cls_accuracy.mean() * 100
        
        tensorboard_log = {'acc': avg_accuracy}
        
        return {
            'loss': loss,
            'log': tensorboard_log
        }
    
    def state_dict(self):
        return self.model.state_dict()
    
    def load_state_dict(self, state_dict):
        self.model.load_state_dict(state_dict)
    

In [16]:
model = LitModel(
    num_classes=20,
    model=model,
    learning_rate=learning_rate,
    momentum=momentum
)

## Callback

In [17]:
# Logging Callback
class MyLoggingCallback(pl.callbacks.Callback):
    def __init__(self, every_batch=100):
        super().__init__()
        self.every_batch = every_batch

    # Train
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, unused=0): 
        # Calculate and log batch accuracy
        self.log('train/loss', outputs['loss'])
        self.log('train/accuracy', outputs['log']['acc'])
    
    # Valid 
    def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, unused=0): 
        # Calculate and log batch accuracy
        self.log('val/loss', outputs['loss'])
        self.log('val/accuracy', outputs['log']['acc'])
        
    # Test
    def on_test_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, unused=0): 
        # Calculate and log batch accuracy
        self.log('test/loss', outputs['loss'])
        self.log('test/accuracy', outputs['log']['acc'])
        
## Visualization Callback 

class VisualizationCallback(pl.callbacks.Callback):
    def get_kernel_weight(self, pl_module):
        # get the first kernel weight
        kernel = pl_module.model.model.conv1.weight.detach().cpu()
        
        # scale the kernel
        kernels = (kernels - kernels.min()) / kernels.max()
        filters = torchvision.utils.make_grid(kernels.clamp(0, 1))
        return filters
    
    def visualize_kernel(self, trainer, pl_module, stage=None):
        if stage=='fit':
            pl_module.logger.experiment.add_image('filters', self.get_kernels(pl_module))
                
            sampleImg=torch.rand((32, 3, 224, 224))
            pl_module.logger.experiment.add_graph(pl_module.model,sampleImg)
        

## Training and Testing

In [ ]:
trainer = pl.Trainer(
    gpus=[0],
    max_epochs=num_epochs,
    logger=logger,
    fast_dev_run=False,
    log_every_n_steps=1,
    callbacks=[MyLoggingCallback(), VisualizationCallback()]
)

trainer.fit(model, data_module)

/home/longvuduc/DS_internship_2022/labtest/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint


Number of labels:  20


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name      | Type          | Params
--------------------------------------------
0 | model     | NeuralNetwork | 23.8 M
1 | criterion | BCELoss       | 0     
--------------------------------------------
23.8 M    Trainable params
0         Non-trainable params
23.8 M    Total params
95.092    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/longvuduc/DS_internship_2022/labtest/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/longvuduc/DS_internship_2022/labtest/lib/python3.8/site-packages/torchvision/transforms/functional.py:150: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:172.)
  img = torch.from_numpy(pic.transpose((2, 0, 1)))

...

Epoch 40, global step 6437: 'val/accuracy' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 41, global step 6594: 'val/accuracy' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 42, global step 6751: 'val/accuracy' was not in top 1


In [ ]:
trainer.test(model, data_module)